# Gene regulatory network analysis in Glioblastoma 
Authors: Camila Lopes-Ramos<sup>1</sup>, Marieke Kuijjer<sup>2</sup>

<sup>1</sup> Department of Biostatistics, Harvard T.H. Chan School of Public Health, Boston, MA.

<sup>2</sup> Centre for Molecular Medicine Norway, University of Oslo, Oslo, Norway.

## Introduction
This tutorial comes with our recent study of gene regulatory networks in Glioblastoma<sup>1</sup>. All the generated networks in this analysis are available in [GRAND database](https://grand.networkmedicine.org/cancers/).
## Load packages

We start first by loading the required packages.

In [ ]:
library(limma) # for differential targeting analysis
library(reshape2) # for data processing
library(ggplot2) # for plotting
library(tidyr)  # for data processing
library(ggpubr)
library(igraph)
library(netZooR) # to load SAMBAR and ALPACA
library(GO.db)
library(GOstats)
library(dplyr)
library(ggbeeswarm)
library(ggrepel)
library(topGO) # for enrichement analysis

## 1. Community structure comparison with ALPACA

Here, we calculate differential communities between the long-term and short-term survival groups using ALPACA on the two discovery datasets. We also perform GO term enrichment analysis to determine whether the differential communities are enriched for biological processes, and output these in a heatmap. Finally, we draw ALPACA scores and highlight top TFs and target genes.

This scripts loads the average individual-patient networks for the short-term and long-term survival groups. To get one network representing each patient subgroup, we took the mean of each edge weight in the single-sample networks of patients belonging to that subgroup (long-term or short-term survival). The complete single-sample networks (>30Gb of data) are available on [GRAND](https://grand.networkmedicine.org).

Finally, please note that different versions of the GO term database may result in slightly different enrichment scores and pathways. In the manuscript, we used R version 3.2.3 and topGO version 2.22.0.

In [ ]:
# functions
# check if ALPACA score of a TF/target gene is an outlier
is_outlier <- function(x){
	return(x!="no")
}
# transform edge weights to positive values
transform_avgnets<-function(n) {
	n<-log(exp(1)^n + 1)
	n
}

# directories
setwd("../data/")
dir.create("data")
dir.create("results")
home_in <- "data"
home_out <- "results"
setwd(home_in)

### 1.1. Run ALPACA comparing communities
Discovery dataset 1

In [ ]:
load("/opt/data/netZooR/gbm/data/dat2_avgnets.RData")

# Filter these networks for canonical edges, which are edges supported by a TFBS in the target gene's promoter
prior_650 <- read.delim("/opt/data/netZooR/gbm/data/motif.txt", header=F)
prior_650<-prior_650[prior_650$V3==1,]
row.names(prior_650)<-paste(prior_650$V1, prior_650$V2)
row.names(avgnets)<-paste(avgnets$reg, avgnets$tar, sep=" ")
avgnets_prior<-avgnets[intersect(row.names(avgnets), row.names(prior_650)),]

# Transform edge weight to positive weights, so that community structure detection can be run
avgnets_prior$avg_gbm0<-(sapply(as.list(avgnets_prior[,3]), transform_avgnets))
avgnets_prior$avg_gbm1<-(sapply(as.list(avgnets_prior[,4]), transform_avgnets))

# Run CONDOR_ALPACA, comparing long- with short-term survival network communities
setwd("..")
setwd(home_out)
avgnets_condor_gbm<-avgnets_prior
names(avgnets_condor_gbm)[1:2]<-c("tf","gene")
dir.create("alpaca")
dat2_alpaca<-alpaca(avgnets_condor_gbm, "alpaca/dat2")

# Run CONDOR_ALPACA, comparing short- with long-term survival network communities
avgnets_condor_gbm<-avgnets_prior[,c(1,2,4,3)] # swap short-term and long-term survival columns
names(avgnets_condor_gbm)[1:2]<-c("tf","gene")
dat2_10_alpaca<-alpaca(avgnets_condor_gbm, "alpaca/dat2_10", verbose=F)

Discovery dataset 2

In [ ]:
setwd("../data")
load("/opt/data/netZooR/gbm/data/dat3_avgnets.RData")

# Filter these networks for canonical edges, which are edges supported by a TFBS in the target gene's promoter
prior_650 <- read.delim("/opt/data/netZooR/gbm/data/motif.txt", header=F)
prior_650<-prior_650[prior_650$V3==1,]
row.names(prior_650)<-paste(prior_650$V1, prior_650$V2)
row.names(avgnets)<-paste(avgnets$reg, avgnets$tar, sep=" ")
avgnets_prior<-avgnets[intersect(row.names(avgnets), row.names(prior_650)),]

# Transform edge weight to positive weights, so that community structure detection can be run
avgnets_prior$avg_gbm0<-(sapply(as.list(avgnets_prior[,3]), transform_avgnets))
avgnets_prior$avg_gbm1<-(sapply(as.list(avgnets_prior[,4]), transform_avgnets))

# Run CONDOR_ALPACA, comparing long- with short-term survival network communities
setwd("..")
setwd(home_out)
avgnets_condor_gbm<-avgnets_prior
names(avgnets_condor_gbm)[1:2]<-c("tf","gene")
dat3_alpaca<-alpaca(avgnets_condor_gbm, "alpaca/dat3")

# Run CONDOR_ALPACA, comparing short- with long-term survival network communities
avgnets_condor_gbm<-avgnets_prior[,c(1,2,4,3)] # swap short-term and long-term survival columns
names(avgnets_condor_gbm)[1:2]<-c("tf","gene")
dat3_10_alpaca<-alpaca(avgnets_condor_gbm, "alpaca/dat3_10", verbose=F)

### 1.2. GO enrichment on differential modules
In this section, we will perform functional enrichment analysis on differential ALPACA<sup>5</sup> modules. However, since this step might take sometime, we can load precomputed results.

In [ ]:
loadprecomupted=1
if loadprecomputed == 1:
    load("/opt/data/netZooR/gbm/GOterm_results.RData")
else:
    toevaluate <- c("dat2", "dat3", "dat2_10", "dat3_10")

    for(t in 1:length(toevaluate)){

        # Load ALPACA scores and final communities
        readscores <- paste("alpaca/", toevaluate[t], "_ALPACA_scores.txt", sep="")
        readcomms <- paste("alpaca/", toevaluate[t], "_ALPACA_final_memb.txt", sep="")
        scores <- read.delim(readscores, header=F)
        comms <- read.delim(readcomms, header=F)
        tosel <- intersect(scores[,1], comms[,1])
        scores <- scores[which(scores[,1] %in% tosel),]
        comms <- comms[which(comms[,1] %in% tosel),]
        scores <- scores[order(scores[,1]),]
        comms <- comms[order(comms[,1]),]
        scores[,1] <- as.character(scores[,1])
        comms[,1] <- as.character(comms[,1])
        all(scores[,1]==comms[,1])
        scores <- cbind(scores, comms[,2])
        row.names(scores) <- scores[,1]
        scores <- scores[,-1]
        colnames(scores) <- c("score", "com")

        # Extract ALPACA scores for each of the target genes
        alpaca_nodes <- row.names(scores)
        table(substr(alpaca_nodes, nchar(alpaca_nodes), nchar(alpaca_nodes))) # 647 A (tf) 10698 B (gene)
        idx <- which(substr(alpaca_nodes, nchar(alpaca_nodes), nchar(alpaca_nodes))=="B")
        alpaca_genes <- scores[idx,]
        alpaca_genes <- alpaca_genes[order(alpaca_genes[,2],decreasing=T),]
        alpaca_genes <- alpaca_genes[,c(2,1)]

        # Extract the top 50 genes in each community
        for(n in 1:max(alpaca_genes$com)){
            junk <- alpaca_genes[which(alpaca_genes$com==n),]
            junk <- junk[order(junk$score,decreasing=T),]
            if(nrow(junk)>50){
                junk <- junk[1:50,]
                if(n==1){
                    newfile <- junk
                } else {
                    newfile <- rbind(newfile, junk)
                }
            }
        }

        # Prepare the background geneset for topGO
        gene_lst<-substr(row.names(newfile), 1, nchar(row.names(newfile))-2)
        newfile <- cbind(newfile, gene_lst)
        alpaca_genes <- cbind(alpaca_genes, substr(row.names(alpaca_genes), 1, nchar(row.names(alpaca_genes))-2))
        colnames(alpaca_genes)[3] <- "genes"

        # GO term enrichment analysis for each community
        GO2Symbol.bp <- annFUN.org(whichOnto = "BP", mapping = "org.Hs.eg.db", ID = "symbol")
        symbol2GO.bp <- inverseList(GO2Symbol.bp)
        enrichRes_comm <- c()
        for(comm_num in as.numeric(names(table(newfile$com)))){
            # select interesting genes and background
            genesel <- as.character(newfile[newfile$com %in% comm_num,]$gene_lst)
            interesting <- factor(as.integer(alpaca_genes$genes %in% genesel))
            names(interesting) <- alpaca_genes$genes
            # topGO results
            tgd2<- new("topGOdata", ontology = "BP", allGenes = interesting, nodeSize = 1 , annot = annFUN.gene2GO, gene2GO = symbol2GO.bp)
            resultFisher = runTest(tgd2, algorithm="classic", statistic="fisher")
            enrichRes.bp <- GenTable(tgd2, classic = resultFisher, orderBy = "classic", ranksOf = "classic", topNodes = length(score(resultFisher)))
            # Keep GO terms with 10-100 genes, output top 10 terms per community
            enrichRes.bp<-(subset(enrichRes.bp, enrichRes.bp$Annotated<100))
            enrichRes.bp<-(subset(enrichRes.bp, enrichRes.bp$Annotated>10))
            adjP_Fisher <- p.adjust(enrichRes.bp[,6],"BH")
            commID<- rep(comm_num, rep(length(enrichRes.bp[,6])))
            enrichRes.bp <- cbind(enrichRes.bp, adjP_Fisher, commID)
            enrichRes_comm<-rbind(enrichRes_comm,enrichRes.bp[1:10,])
            print(comm_num)
        }

        # Store significant results
        govar <- paste("enrichRes_", toevaluate[t], sep="")
        assign(govar, enrichRes_comm)

        cat("Done running ALPACA on:", toevaluate[t], "\n\n")

    } # end loop over t

### 1.3. Visualize GO term enrichment results

In [ ]:
prepare_go_per_datatype <- function(x, dataset="D1"){
	colnames(x)[ncol(x)] <- "Community"	
	x$Community <- paste(dataset, x$Community, sep="_")
	x <- x[which(x$adjP_Fisher<0.25 & x$Annotated<50),]
	signexp <- x$Significant/x$Expected
	x <- cbind(x, signexp)
	return(x)
}
res2 <- prepare_go_per_datatype(enrichRes_dat2, dataset="D1")
res2_10 <- prepare_go_per_datatype(enrichRes_dat2_10, dataset="D1")
res3 <- prepare_go_per_datatype(enrichRes_dat3, "D2")
res3_10 <- prepare_go_per_datatype(enrichRes_dat3_10, "D2")

Prepare both datasets into a dataframe for visualization in a heatmap

In [ ]:
prepare_go_vis <- function(x, y){
	# combine both datasets
	dat <- rbind(x,y)
	dat <- dat[order(dat$signexp, decreasing=T),]
	dat <- dat[rev(1:nrow(dat)),]
	dat$Term <- factor(dat$Term, levels = unique(dat$Term))
	# add annotation on which of the discovery datasets the community belongs to
	dataset <- substr(dat$Community,1,2)
	dat <- cbind(dat, dataset)
	dat[which(dat$dataset=="D1"),]$signexp <- -dat[which(dat$dataset=="D1"),]$signexp
	dat <- cbind(dat, -log(dat$adjP_Fisher,10))
	colnames(dat)[ncol(dat)] <- "-logFDR"
    statistic <- dat$"-logFDR"*dat$signexp
	dat <- cbind(dat, statistic)
	dat$signexp[dat$signexp<(-40)] <- -40 # use a maximum threshold for color scale
	dat$signexp[dat$signexp>(40)] <- 40 # use a maximum threshold for color scale
	return(dat)
}
toplot_01 <- suppressWarnings(prepare_go_vis(res2, res3))
toplot_10 <- suppressWarnings(prepare_go_vis(res2_10, res3_10))

Visualize enriched GO terms in the long-term vs short-term community structure comparison. 

First, we change community names for plotting. The order is arbitrary and based on the entire differential community structure analysis, which detects more communities. Here, we only want to visualize those communities with significant GO term enrichment, so we will change these names to M1, M2, etc for clarity

In [ ]:
comname <- toplot_01$Community
comname[which(comname=="D1_10")] <- "D1_M2"
comname[which(comname=="D1_11")] <- "D1_M1"
comname[which(comname=="D2_10")] <- "D2_M1"
comname[which(comname=="D2_13")] <- "D2_M2"
comname[which(comname=="D2_16")] <- "D2_M3"
toplot_01$Community <- comname

Sort and plot

In [ ]:
newlevelorder <- toplot_01[,c(2,8,9)]
newlevelorder$signexp <- abs(newlevelorder$signexp)
newlevelorder <- newlevelorder[order(newlevelorder$signexp, decreasing=T),]
newlevelorder <- newlevelorder[order(newlevelorder$Community),]
newlevelorder <- as.character(unique(newlevelorder$Term))
newlevelorder <- rev(newlevelorder)

In [ ]:
#	pdf("heatmap_long_vs_short_term.pdf", w=6, h=3.5)
ggplot(toplot_01, aes(Community, factor(Term, level=newlevelorder), fill=signexp))+
    scale_fill_gradient2(low="purple", high="darkgreen", limits=c(-40,40))+
    theme_bw()+
    labs(x="Differential Module", y="")+
    geom_tile()
# dev.off()

Now, we visualize enriched GO terms in the short-term vs long-term community structure comparison.

First,we change community names for plotting (see above).

In [ ]:
comname <- toplot_10$Community
comname[which(comname=="D1_13")] <- "D1_M1"
comname[which(comname=="D1_15")] <- "D1_M2"
comname[which(comname=="D2_11")] <- "D2_M1"
comname[which(comname=="D2_14")] <- "D2_M2"
toplot_10$Community <- comname

Sort and plot

In [ ]:
newlevelorder <- toplot_10[,c(2,8,9)]
newlevelorder$signexp <- abs(newlevelorder$signexp)
newlevelorder <- newlevelorder[order(newlevelorder$signexp, decreasing=T),]
newlevelorder <- newlevelorder[order(newlevelorder$Community),]
newlevelorder <- as.character(unique(newlevelorder$Term))
newlevelorder <- rev(newlevelorder)

In [ ]:
#	pdf("heatmap_long_vs_short_term.pdf", w=6, h=3.5)
ggplot(toplot_10, aes(Community, factor(Term, level=newlevelorder), fill=signexp))+
	scale_fill_gradient2(low="purple", high="darkgreen", limits=c(-40,40))+
	theme_bw()+
	labs(x="Differential Module", y="")+
	geom_tile()
# dev.off()

## 1.4. Visualize top TFs and target genes based on ALPACA scores

Load ALPACA scores and final communities

In [ ]:
toreadscores <- function(x){
	readscores <- paste("alpaca/", x, "_ALPACA_scores.txt", sep="")
	readcomms <- paste("alpaca/", x, "_ALPACA_final_memb.txt", sep="")
	scores <- read.delim(readscores, header=F)
	comms <- read.delim(readcomms, header=F)
	tosel <- intersect(scores[,1], comms[,1])
	scores <- scores[which(scores[,1] %in% tosel),]
	comms <- comms[which(comms[,1] %in% tosel),]
	scores <- scores[order(scores[,1]),]
	comms <- comms[order(comms[,1]),]
	scores[,1] <- as.character(scores[,1])
	comms[,1] <- as.character(comms[,1])
	all(scores[,1]==comms[,1])
	scores <- cbind(scores, comms[,2])
	row.names(scores) <- scores[,1]
	scores <- scores[,-1]
	colnames(scores) <- c("score", "com")
	return(scores)
}
scores2_dat <- toreadscores("dat2")
scores3_dat <- toreadscores("dat3")
scores2_10_dat <- toreadscores("dat2_10")
scores3_10_dat <- toreadscores("dat3_10")

In [ ]:
prepare_vioplot <- function(scores2,scores3){

	# Intersect TFs and genes in both datasets
	scores2 <- scores2[order(row.names(scores2)),]
	scores3 <- scores3[order(row.names(scores3)),]
	namesboth <- intersect(row.names(scores2), row.names(scores3))
	scores2 <- scores2[which(row.names(scores2) %in% namesboth),]
	scores3 <- scores3[which(row.names(scores3) %in% namesboth),]
	all(row.names(scores2)==row.names(scores3))

	# Select scores for TFs and target genes
	a <- row.names(scores2)
	a <- substr(a,nchar(a),nchar(a))
	tfidx <- which(a=="A")
	geneidx <- which(a=="B")

	# Calculate the average ALPACA scores for discovery dataset 1 and 2
	tf2 <-scores2[tfidx,c(2,1)]
	tf3 <-scores3[tfidx,c(2,1)]
	gene2 <-scores2[geneidx,c(2,1)]
	gene3 <-scores3[geneidx,c(2,1)]
	tfmix <- cbind(tf2,tf3[,2])
	tfmix <- cbind(tfmix, apply(tfmix[,2:3], 1, mean))
	colnames(tfmix) <- c("TF","dat2score","dat3score","meanscore")
	tfmix[,1] <- as.character(tfmix[,1])
	tfmix[,1] <- substr(row.names(tfmix), 1, nchar(row.names(tfmix))-2)
	genemix <- cbind(gene2,gene3[,2])
	genemix <- cbind(genemix, apply(genemix[,2:3],1,mean))
	colnames(genemix) <- c("Gene","dat2score","dat3score","meanscore")
	genemix[,1] <- as.character(genemix[,1])
	genemix[,1] <- substr(row.names(genemix), 1, nchar(row.names(genemix))-2)
	tfmix <- tfmix[order(tfmix$meanscore, decreasing=F),]
	genemix <- genemix[order(genemix$meanscore, decreasing=F),]
	tfmix <- tfmix[order(tfmix$meanscore, decreasing=T),]
	genemix <- genemix[order(genemix$meanscore, decreasing=T),]

	# Select TFs and genes to highlight based on threshold (1.5xIQR for TFs, 3xIQR for target genes)
	tflog <- log(tfmix$meanscore,10)
	tflogthres <- median(tflog)+1.5*IQR(tflog)
	genelogorig <- log(genemix$meanscore,10)
	genelog <- genelogorig[which(!is.na(genelogorig))] # there are a few NAs as some of these values are very close to 0, remove these
	genelogthres <- median(genelog)+3*IQR(genelog)
	genemixnona <- genemix[which(!is.na(genelogorig)),] # remove NAs from the original data as well

	# Flag significant TFs and target genes for visualization purposes
	tfnames <- tfmix$TF
	genenames <- genemix$Gene
	tfsign <- rep("no", length(tfnames))
	tfsign[which(tflog>tflogthres)] <- "sign"
	genesign <- rep("no", length(genenames))
	genesign[which(genelogorig>genelogthres)] <- "sign"

	# Plot
	viodat <- rbind(cbind(tflog, "TF", tfnames, tfsign), cbind(genelog, "Gene", genenames, genesign))
	viodat <- as.data.frame(viodat)
	colnames(viodat) <- c("score","type","node","sign")
	viodat[,1] <- as.numeric(as.character(viodat[,1]))
	viodat2 <- as.matrix(viodat)
	idx <- viodat2[,4]=="sign"
	viodat2[idx,4] <- viodat2[idx,3]
	viodat2[!idx,4] <- ""
	viodat2 <- as.data.frame(viodat2)
	viodat2[,1] <- as.numeric(as.character(viodat2[,1]))
	viodat2[,2] <- factor(viodat2[,2], levels=c("TF","Gene"))

    return(viodat2)

}

Plot ALPACA scores of the long-term vs short-term comparison

In [ ]:
viodat2 <- prepare_vioplot(scores2=scores2_dat, scores3=scores3_dat)
#pdf("vioplot_long_vs_shortterm.pdf", w=12, h=8)
	viodat2 %>%
		group_by(type) %>%
            mutate(outlier = ifelse(is_outlier(sign), node, as.numeric(NA))) %>%
                ggplot(mapping=aes(type, score))+
                    geom_quasirandom(aes(color=viodat2$type))+
					geom_text_repel(aes(label = sign), size = 5, box.padding = unit(0.35, "lines"), point.padding = unit(0.3, "lines"))+
					theme_classic(base_size = 25)+
					scale_color_manual(values=c("#E69F00", "#56B4E9"))+
					theme(legend.position = "none")+
					labs(x="", y="Log10 diff. modularity score")
	#dev.off()

Plot ALPACA scores of the short-term vs long-term comparison

In [ ]:
viodat2 <- prepare_vioplot(scores2=scores2_10_dat, scores3=scores3_10_dat)
#pdf("vioplot_short_vs_longterm.pdf", w=12, h=8)
	viodat2 %>%
		group_by(type) %>%
			mutate(outlier = ifelse(is_outlier(sign), node, as.numeric(NA))) %>%
                ggplot(mapping=aes(type, score))+
                    geom_quasirandom(aes(color=viodat2$type))+
					geom_text_repel(aes(label = sign), size = 5, box.padding = unit(0.35, "lines"), point.padding = unit(0.3, "lines"))+
					theme_classic(base_size = 25)+
					scale_color_manual(values=c("#E69F00", "#56B4E9"))+
					theme(legend.position = "none")+
					labs(x="", y="Log10 diff. modularity score")
	#dev.off()

## 2. Differential regulation analysis

Here, we will perform a differential regulation analysis using LIMMA, and then perform Gene Set Enrichment Analysis to identify pathways differentially regulated between long- and short-term survivors. We will do this first for the discovery datasets and then for the validation dataset.

In [ ]:
# Functions
'%ni%' <- Negate('%in%')

Load data

In [ ]:
load("/opt/data/netZooR/gbm/allpats_gbm0_gbm1.RData")
load("/opt/data/netZooR/gbm/clinfull.RData")

General settings

In [ ]:
datatype <- c("dat2", "dat3")
covariates <- "ageneosex"

GSEA settings

In [ ]:
escut <- 0.5
fdrcut <- 0.05
sizecut <- 50

signaturename <- "/opt/data/netZooR/gbm/c2.cp.reactome.v5.0.symbols.gmt" # signature db you want to test
signnamelength <- 8 # set to remove prefix from signature names (8 for "REACTOME")
charcut <- 40 # cut signature name in heatmap to this nr of characters
nriter <- 1000 # number of iterations in GSEA
dencol_neg <- "red" # bubble plot color for negative ES
dencol_pos <- "blue" # bubble plot color for positive ES
asp <- 3 # aspect ratio of bubble plot

Settings to generate a symmetric color key for the bubble plot

In [ ]:
breaksdist <- 0.05 # distance of breaks in the values defining the color palette
fdrcutheat <- -log(0.25,10) # values above this threshold will be plotted in white
maxrange <- 4 # absolute values above this threshold will visualized with the same color
allrange <- maxrange*2
breaks1 <- c(seq(from=-maxrange, to=maxrange, by=breaksdist))
breakscut <- log(fdrcutheat, 10)
breakscut <- round(breakscut/breaksdist)*breaksdist # round so that it becomes part of the range
breakslen <- length(which(breaks1<breakscut))+1
mypalette1a <- colorRampPalette(c("blue","white"), space="rgb")(breakslen)
mypalette1b <- colorRampPalette(c("white","red"), space="rgb")(breakslen)
mypalette1w <- rep("#FFFFFF", length(breaks1)-2*breakslen-1) # white
mypalette1 <- c(mypalette1a,mypalette1w,mypalette1b)

### 2.1. Differential targeting analysis with LIMMA on the discovery datasets

Here, we will perform a LIMMA analysis to compare gene targeting scores in long-term survivors with those in short-term survivors, correcting for the patient's age at diagnosis of the primary tumor, their sex, and whether they were treated with neo-adjuvant therapy or not. We will do this for the two discovery datasets and will save the t-statistics from the LIMMA toptable in a .rnk file that can be used as input for GSEA

In [ ]:
for(d in 1:length(datatype)){
	# load the gene targeting scores (degrees)
		toload <- paste("/opt/data/netZooR/gbm/", datatype[d], "_degree.RData", sep="")
		load(toload)

	# prepare data for limma
		samples <- colnames(indegree)
		samples <- samples[which(samples %in% row.names(clinfull))]
		samples <- samples[which(samples %in% c(gbm0, gbm1))]
		subsamples <- row.names(clinfull[which(!is.na(clinfull$yearstobirth) & !is.na(clinfull$yearstobirth) & !is.na(clinfull$neoadjuvanttherapy) & !is.na(clinfull$gender)),])
		subsamples <- samples[which(samples %in% subsamples)]

	# prepare data and limma groups
		group0 <- gbm0[which(gbm0 %in% subsamples)]
		group1 <- gbm1[which(gbm1 %in% subsamples)]
	    targets0 <- cbind(group0, "group0")
	    targets1 <- cbind(group1, "group1")
	    targets <- rbind(targets0, targets1)
	    targets <- targets[order(targets[,1]),]
	    group <- factor(targets[,2])
		dat <- indegree[,which(colnames(indegree) %in% targets[,1])]
		dat <- dat[,order(as.character(colnames(dat)))]

	# make the limma design matrix
		clinsel <- clinfull[which(row.names(clinfull) %in% targets[,1]),]
		gender <- as.factor(as.character(clinsel$gender))
		neo <- as.factor(as.character(clinsel$neoadjuvanttherapy))
		age <- as.numeric(as.character(clinsel$yearstobirth))
		design <- model.matrix(~group+age+neo+gender)
	
	# check if the minimum group size includes >10 samples, if so, run limma
		tocheck <- design[,which(colnames(design)!="age")]
		mingroup <- rep(NA, length=(ncol(tocheck)-1))
		for(e in 2:ncol(tocheck)){
			mingroup[e-1] <- min(table(tocheck[,e]))
		}
		if(min(mingroup)>10){
			fit <- eBayes(lmFit(dat, design))
			toptable <- topTable(fit, coef="groupgroup1", number=Inf)
			toptable <- toptable[order(row.names(toptable)),]
			topfile <- paste("toptable", datatype[d], covariates, sep="_")
			assign(topfile, toptable)
			rnkdat <- toptable
			rnkdat <- rnkdat[order(abs(rnkdat$t),decreasing=T),]
			rnkfile <- paste("../data/", datatype[d], ".rnk", sep="")
			write.table(cbind(row.names(rnkdat),rnkdat$t), rnkfile, sep="\t", quote=F, row.names=F, col.names=F)

		} else {
			cat(paste(datatype[d], covariates, ": covariate group size too small \n", sep="") )
		}# end if mingroup>10

	print(d)
} # end loop over datatype

### 2.2. GSEA on the discovery datasets

Now, we will run GSEA analysis. We will output the pathways that are significantly differentially regulated in both datasets

In [ ]:
currPath=getwd()
setwd("../data/")
dir.create("gsea", showWarnings=F) # list dirs with "rnk"
rnkfiles <- list.files(pattern="rnk")
for(rnkfile in rnkfiles){
    gseaname <- substr(rnkfile, 1, nchar(rnkfile)-4)
    assign(gseaname, paste("java -Xmx4096m -cp ../gsea2-2.0.13.jar xtools.gsea.GseaPreranked -gmx ../", signaturename, " -collapse false -mode Max_probe -norm meandiv -nperm ", nriter, " -rnk ", rnkfile, " -scoring_scheme weighted -rpt_label ", gseaname, " -include_only_symbols true -make_sets true -plot_top_x 0 -rnd_seed timestamp -set_max 250 -set_min 1 -zip_report false -out gsea/ -gui false", sep=""))
    system(eval(parse(text=gseaname)))
} # end loop over rnkfiles

### 2.3. Make bubble plot

We will now make a bubble plot to visualize the intersection of the pathways identified in both discovery datasets (Figure 3<sup>1</sup>). Note that GSEA is not deterministic and may therefore return slightly different pathways each time it is run. Below, we will provide the original files from GSEA, but also provide code to plot based on the newly run GSEA

In [ ]:
# setwd("gsea") # If you want to run the plot on the newly run GSEA analysis, uncomment this line and comment out the next
setwd("/opt/data/netZooR/gbm/gsea_original_reports") # This loads the results from the original GSEA
gseadirlist <- list.dirs(recursive=F)

for(d in 1:length(gseadirlist)){
  setwd(gseadirlist[d])
  gseafiles <- list.files(pattern="gsea_report") # gsea outputs .html and .xls files for signatures with positive and negative enrichment scores
  neg <- read.delim(gseafiles[2]) # signatures with negative enrichment scores
  pos <- read.delim(gseafiles[4]) # signatures with positive enrichment scores
  gsea <- rbind(neg, pos) # combine the signatures
  dat <- gsea[,c(1,4,5,8)] # keep these columns (contain signature name, size, nominal p-value, and FDR)
  dat <- dat[order(as.character(dat[,1])),] # order the signatures alphabetically
  signsignatures <- as.character(dat[which(dat$FDR.q.val<fdrcut & abs(dat$ES)>escut & dat$SIZE<sizecut),1]) # significant signatures in this dataset
  datsign <- dat[which(dat$FDR.q.val<fdrcut & abs(dat$ES)>escut & dat$SIZE<sizecut),]

  analysisname <- gseadirlist[d]
  datname <- strsplit(analysisname, "/")
  datname <- datname[[1]][length(datname[[1]])-3]
  analysisname <- strsplit(analysisname, "\\.")[[1]][2]
	analysisname <- substr(analysisname, 2, nchar(analysisname))

	datsignname <- paste(analysisname, "sign", sep="")
	assign(datsignname, datsign)

  cat(datname, analysisname, ":", length(signsignatures), "\n")
  if(d==1){
  	intsign <- signsignatures
  } else {
  	intsign <- intersect(intsign, signsignatures)
  }
  cat("\n")
  setwd("..")
}
cat("Significant signatures in both discovery datasets:\n")
print(intsign)

Prepare the intersection of the significant results for the double bubble plot

In [ ]:
test2 <- dat2sign[which(dat2sign$NAME %in% intsign),]
test3 <- dat3sign[which(dat3sign$NAME %in% intsign),]
all(test2$NAME==test3$NAME)
dat <- cbind(test2, test3[,2:4])

Substitute the row names with something more readable

In [ ]:
a <- as.character(dat[,1]) 
for (j in 1:length(a)){
      a[j] <- substr(a[j], signnamelength+2, nchar(a[j]))
}
a <- tolower(a)
for (j in 1:length(a)){
      if(nchar(a[j])>charcut) { a[j] <- paste(substr(a[j], 1, charcut), "...", sep=" ")}
}
a <- gsub("_", " ", a)
dat[,1] <- a

Prepare the data in the right format for plotting with ggplot2

In [ ]:
datnew <- dat[which(dat$ES<0),] # In the original GSEA analysis, there were only 7 pathways, all highly targeted in the long-term survival group (negative ES score). However, GSEA is not deterministic, and rerunning it may return slightly different pathways. Here, we will remove those highly targeted in the short-term survival group for simplicity
datnew <- datnew[order(datnew[,4]),] # FDR.q.val
datnew$signature <- factor(datnew$NAME, rev(as.character(datnew$NAME)))
datnew2 <- datnew[,c(1:4,8)]
datnew3 <- datnew[,c(1,5:8)]
datnew <- rbind(datnew2, datnew3)

Log transform the FDR and give it a sign based on the direction of the differential regulation

In [ ]:
fdrnew <- -log(datnew$FDR.q.val, 10)
fdrnew[which(fdrnew>5)] <- 5 # set to 5 for visualization purposes
fdrnew[which(datnew$ES<0)] <- -(fdrnew[which(datnew$ES<0)])
datnew$FDR.q.val <- fdrnew
colnames(datnew)[4] <- "logFDR"
datnewsub <- datnew[1:(nrow(datnew)/2),]
datnew <- datnew[rev(1:nrow(datnew)),] # reverse these labels
datnew$signature <- factor(datnew$signature, as.character(datnew$signature[1:(nrow(datnew)/2)]))

Here we will sort the rows to visually improve the bubble plot

In [ ]:
d1 <- datnewsub[which(datnewsub$ES<0),]
d2 <- datnewsub[which(datnewsub$ES>0),]

d1 <- d1[order(d1$ES),]
d2 <- d2[order(d2$ES, decreasing=T),]
d <- rbind(d1,d2)

datnewsub <- datnew[1:(nrow(datnew)/2),]
dsuborder <- d[,1]
dorder <- rep(NA, length(dsuborder))
for(i in 1:length(dorder)){
    dorder[i] <- which(datnewsub[,1]==dsuborder[i])
}

datnew7 <- datnew[c(dorder, dorder+(nrow(datnew)/2)),]
datnew7 <- datnew7[rev(1:nrow(datnew7)),]
datnew7$signature <- factor(datnew7$signature, as.character(datnew7$signature[1:(nrow(datnew7)/2)]))

datnewinv <- datnew7
datnewinv$ES <- -datnewinv$ES
datnewinv$logFDR <- -datnewinv$logFDR
l1 <- 0.18; l2 <- 0.25 # position of legend in plot  

Render the bubble plot

In [ ]:
#pdf("bubble_plot.pdf", h=4, w=12)
g<-ggplot(datnewinv, aes(x=ES,y=signature,size=SIZE))
    print(g+geom_point(aes(fill=logFDR), shape=21, colour="white")+
    theme_bw()+ # white background, needs to be placed before the "signcol" line
    xlim(0,1)+ # OBS! match sign01
    scale_size_area(max_size=10,guide="none")+
    #scale_fill_gradient2(low=dencol_neg, high=dencol_pos, limits=c(-3.1,3.1))+
    scale_fill_gradient2(low="white", high=dencol_pos, limits=c(0,3)+
    theme(axis.text.y = element_text(size=rel(1.6)))+
    theme(aspect.ratio=1.5, axis.title.y=element_blank(), legend.position=c(l1,l2)) # test
    ) # rel: relative y-axis text size
)
#dev.off()

Finally, the [Cytoscape](https://cytoscape.org/) file to reproduce the network on the discovery dataset (Figure 4<sup>1</sup>) is hosted at Zenodo, [DOI: 10.5281/zenodo.4651672](https://zenodo.org/record/4651672#.YGYSLBNKiL8).

## 2.4. Differential targeting analysis with LIMMA on the validation dataset
Here, we will run differential targeting analysis with LIMMA on the validation dataset from the German Glioma Network, comparing the short-term with the long-term survival group

Load the clinical data from the German Glioma Network

In [ ]:
clinfull <- read.delim("/opt/data/netZooR/gbm/survival_info.txt", header=F)
idxpoor <- grep("short-term", clinfull[,2])
idxmed <- grep("intermediate", clinfull[,2])
idxgood <- grep("long-term", clinfull[,2])
clinfull <- as.matrix(clinfull)
clinfull[idxpoor,2] <- "poor"
clinfull[idxmed,2] <- "intermediate"
clinfull[idxgood,2] <- "good"

Load the gene targeting scores (degrees)

In [ ]:
datatype <- c(datatype, "ggn"); d<-3;
toload <- paste("/opt/data/netZooR/gbm/", datatype[d], "_degree.RData", sep="")
load(toload)

Prepare input data for LIMMA analysis

In [ ]:
analysistype <- "poorintgood"
group0 <- clinfull[which(clinfull[,2]=="good"),1]
groupm <- clinfull[which(clinfull[,2]=="intermediate"),1]
group1 <- clinfull[which(clinfull[,2]=="poor"),1]
targets0 <- cbind(group0, "group0")
targetsm <- cbind(groupm, "groupm")
targets1 <- cbind(group1, "group1")
inputdata <- "indegree"
datatouse <- eval(parse(text=inputdata))
targets <- rbind(targets0, targetsm, targets1)
targets <- targets[order(targets[,1]),]
group <- factor(targets[,2])
dat <- datatouse[,which(colnames(datatouse) %in% targets[,1])]
dat <- dat[,order(as.character(colnames(dat)))]
clinsel <- clinfull[which(clinfull[,1] %in% targets[,1]),]

Perform LIMMA analysis

In [ ]:
design <- model.matrix(~0+group)
contrast.matrix <- makeContrasts(groupgroup1-groupgroup0, groupgroup1-groupgroupm, groupgroupm-groupgroup0, levels=design)
fit <- lmFit(dat, design)
fit2 = contrasts.fit(fit, contrast.matrix)
fit2e = eBayes(fit2)
toptable <- topTable(fit2e, number=Inf)
toptable10 <- topTable(fit2e, coef=1, number=Inf) # compare short-term with long-term survival

Save t-statistic in .rnk file for GSEA

In [ ]:
setwd(currPath)
rnkdat10 <- cbind(row.names(toptable10), toptable10$t)
rnkname10 <- paste("../data/ggn.rnk")
write.table(rnkdat10, rnkname10, sep="\t", quote=F, row.names=F, col.names=F)

## 2.5. GSEA on the validation dataset

We will now run GSEA on the ranked t-statistics obtained from the LIMMA analysis above

In [ ]:
setwd("../data/")
dir.create("gsea", showWarnings=F) # list dirs with "rnk"
rnkfiles <- list.files(pattern="rnk")
rnkfile <- rnkfiles[3]
gseaname <- substr(rnkfile, 1, nchar(rnkfile)-4)
assign(gseaname, paste("java -Xmx4096m -cp ../gsea2-2.0.13.jar xtools.gsea.GseaPreranked -gmx ../", signaturename, " -collapse false -mode Max_probe -norm meandiv -nperm ", nriter, " -rnk ", rnkfile, " -scoring_scheme weighted -rpt_label ", gseaname, " -include_only_symbols true -make_sets true -plot_top_x 0 -rnd_seed timestamp -set_max 250 -set_min 1 -zip_report false -out gsea/ -gui false", sep=""))
system(eval(parse(text=gseaname)))

Note that GSEA is not deterministic and may therefore return slightly different pathways each time it is run. Below, we will provide the original files from GSEA, but also provide code to plot based on the newly run GSEA

In [ ]:
setwd("/opt/data/netZooR/gbm/gsea_original_reports_validation") # This loads the results from the original GSEA
gseadirlist <- list.dirs(recursive=F)
d=1

In [ ]:
setwd(gseadirlist[d])
gseafiles <- list.files(pattern="gsea_report") # gsea outputs .html and .xls files for signatures with positive and negative enrichment scores
neg <- read.delim(gseafiles[2]) # signatures with negative enrichment scores
pos <- read.delim(gseafiles[4]) # signatures with positive enrichment scores
gsea <- rbind(neg, pos) # combine the signatures
dat <- gsea[,c(1,4,5,8)] # keep these columns (contain signature name, size, nominal p-value, and FDR)
dat <- dat[order(as.character(dat[,1])),] # order the signatures alphabetically
signsignatures <- as.character(dat[which(dat$FDR.q.val<fdrcut & abs(dat$ES)>escut & dat$SIZE<sizecut),1]) # significant signatures in this dataset
datsign <- dat[which(dat$FDR.q.val<fdrcut & abs(dat$ES)>escut & dat$SIZE<sizecut),]

Output those signatures that are significant in both discovery datasets (from the bubble plot drawn previously) and the validation dataset

In [ ]:
cat("Signatures that are significant in the discovery datasets as well as the validation dataset:\n")
print(as.character(datsign$NAME[which(datsign$NAME %in% intsign)]))

## 3. Estimation of cell compositions
This is code for Supplemental Figure S1<sup>1</sup>.
We used xCell<sup>3</sup> to estimate cell compositions in each tumor sample in each of the three datasets. The scores calculated by xCell approximate cell type fractions, and adjust for overlap between closely related cell types. We applied the xCell pipeline (https://xcell.ucsf.edu) to each of the datasets and used the default threshold of 0.2 to filter out cell types not present in the datasets.
We performed t-test followed by p-value adjustment by FDR to compare xCell score values between short and long-term survival groups.
Figure S1 shows boxplots comparing xCell score values between short and long-term survival groups in the discovery and validation datasets. Here we included only cell types with significant abundance levels (xCell scores > 0.2 in at least one sample) as well as total immune and microenvironment scores.
## 3.1. Data processing
#### Supplemental Figure S1.A - Discovery dataset 1

In [ ]:
setwd(currPath) # get back to user directory
# Load xCell scores data
dat1_xcell <- read.delim("/opt/data/netZooR/gbm/xCell_score_discoverySet1.txt",stringsAsFactors = F, check.names = F)
# Keep only cell types present with an xCell score of at least 0.2 in at least 1 sample
idx <-  apply(dat1_xcell, 2, function(x) sum(x > 0.2) >= 1 )
dat1_xcellsub <- dat1_xcell[,idx]
# Calculate adjusted p-values
a1 <- which(dat1_xcellsub$survival=="long")
a2 <- which(dat1_xcellsub$survival=="short")
pvals <-  apply(dat1_xcellsub[,1:ncol(dat1_xcellsub)-1], 2, function(x) t.test(x[a1],x[a2])$p.val )
adjp <- p.adjust(pvals)
# Cell types with significantly different abundance between long- and short-term survival
# CD8+ naive T-cells FDR=0.01491527
adjp[which(adjp<0.05)] 
# Boxplot comparing xCell score values between long- and short-term survival
dat1_xcellsub_plot <- melt(dat1_xcellsub)
colnames(dat1_xcellsub_plot)[2] <- "celltype"
ggplot(dat1_xcellsub_plot, aes(x = reorder(celltype, -value, FUN = median), y = value, fill = survival))  + geom_boxplot()+ theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust=.3)) +scale_fill_manual(values=c("dodgerblue", "firebrick1")) +labs(title = "Discovery dataset 1", x = "Cell Type", y= "xCell Value") #ordered x axis descending

#### Supplemental Figure S1.B - Discovery dataset 2

In [ ]:
# Load xCell scores data
dat2_xcell <- read.delim("/opt/data/netZooR/gbm/xCell_score_discoverySet2.txt",stringsAsFactors = F, check.names = F)
# Keep only cell types present with an xCell score of at least 0.2 in at least 1 sample
idx <-  apply(dat2_xcell, 2, function(x) sum(x > 0.2) >= 1 )
dat2_xcellsub <- dat2_xcell[,idx]
# Calculate adjusted p-values
a1 <- which(dat2_xcellsub$survival=="long")
a2 <- which(dat2_xcellsub$survival=="short")
pvals <-  apply(dat2_xcellsub[,1:ncol(dat2_xcellsub)-1], 2, function(x) t.test(x[a1],x[a2])$p.val )
adjp <- p.adjust(pvals)
# Cell types with significantly different abundance between long- and short-term survival
# CD8+ naive T-cells FDR=0.07766112
# CD4+ memory T-cells FDR=0.09203959
adjp[which(adjp<0.1)] 
# Boxplot comparing xCell score values between long- and short-term survival
dat2_xcellsub_plot <- melt(dat2_xcellsub)
colnames(dat2_xcellsub_plot)[2] <- "celltype"
ggplot(dat2_xcellsub_plot, aes(x = reorder(celltype, -value, FUN = median), y = value, fill = survival))  + geom_boxplot()+ theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust=.3)) +scale_fill_manual(values=c("dodgerblue", "firebrick1")) +labs(title = "Discovery dataset 2", x = "Cell Type", y= "xCell Value") #ordered x axis descending

#### Supplemental Figure S1.C - Validation dataset

In [ ]:
# Load xCell scores data
dat3_xcell <- read.delim("/opt/data/netZooR/gbm/xCell_score_validationSet.txt",stringsAsFactors = F, check.names = F)
# Keep only cell types present with an xCell score of at least 0.2 in at least 1 sample
idx <-  apply(dat3_xcell, 2, function(x) sum(x > 0.2) >= 1 )
dat3_xcellsub <- dat3_xcell[,idx]
# Calculate adjusted p-values
a1 <- which(dat3_xcellsub$survival=="long")
a2 <- which(dat3_xcellsub$survival=="short")
pvals <-  apply(dat3_xcellsub[,1:ncol(dat3_xcellsub)-1], 2, function(x) t.test(x[a1],x[a2])$p.val )
adjp <- p.adjust(pvals)
# Cell types with significantly different abundance between long- and short-term survival
# No cell types found with FDR<0.1
adjp[which(adjp<0.1)] 
# Boxplot comparing xCell score values between long- and short-term survival
dat3_xcellsub_plot <- melt(dat3_xcellsub)
colnames(dat3_xcellsub_plot)[2] <- "celltype"
ggplot(dat3_xcellsub_plot, aes(x = reorder(celltype, -value, FUN = median), y = value, fill = survival))  + geom_boxplot()+ theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust=.3)) +scale_fill_manual(values=c("dodgerblue", "firebrick1")) +labs(title = "Validation dataset", x = "Cell Type", y= "xCell Value") #ordered x axis descending

## 3.2. Comparing short and long-term survival groups

This is the code for Supplemental Figure S2<sup>1</sup>.

In [ ]:
# Load xCell scores data
dat1_xcell <- read.delim("/opt/data/netZooR/gbm/xCell_score_discoverySet1.txt",stringsAsFactors = F, check.names = F)
dat2_xcell <- read.delim("/opt/data/netZooR/gbm/xCell_score_discoverySet2.txt",stringsAsFactors = F, check.names = F)
dat3_xcell <- read.delim("/opt/data/netZooR/gbm/xCell_score_validationSet.txt",stringsAsFactors = F, check.names = F)

In [ ]:
# Load indegree for PD1 pathway genes
# The PD1 pathway targeting score is the average indegree for genes in the PD1 pathway.
dat1_pd1_deg <- read.delim("/opt/data/netZooR/gbm/indegree_pd1genes_discoverySet1.txt", stringsAsFactors = F)
dat2_pd1_deg <- read.delim("/opt/data/netZooR/gbm/indegree_pd1genes_discoverySet2.txt", stringsAsFactors = F)
dat3_pd1_deg <- read.delim("/opt/data/netZooR/gbm/indegree_pd1genes_validationSet.txt", stringsAsFactors = F)

A) Boxplots comparing the distributions of immune scores from xCell between short- and long-term survival groups in the discovery and validation datasets.

In [ ]:
# A) Boxplots comparing the distributions of immune scores from xCell between short- and long-term survival groups in the discovery and validation datasets.
boxplot_one_cellType<-function(xcell_exp,cellType,datasetName){
  xcell <- xcell_exp %>% gather (key = cell_type, value = xcell_value, cellType)
  xcell$xcell_value <-as.numeric(xcell$xcell_value)
  xcell$cell_type <- as.factor(xcell$cell_type)
  xcell$survival <- sub("long","long-term",xcell$survival)
  xcell$survival <- sub("short","short-term",xcell$survival)
  xcell$survival <- factor(xcell$survival, levels=c("long-term", "short-term"))
  plot<-ggboxplot(xcell, x = "survival",y= "xcell_value", fill = "survival",palette = c("dodgerblue", "firebrick1")) + labs(title = datasetName, x = "Survival", y= "xCell Immune Score")
  plot
}

immune_score_box1<-boxplot_one_cellType(dat1_xcell, "ImmuneScore", "Discovery dataset 1")
immune_score_box2<-boxplot_one_cellType(dat2_xcell, "ImmuneScore", "Discovery dataset 2")
immune_score_box3<-boxplot_one_cellType(dat3_xcell, "ImmuneScore", "Validation dataset")

B) Scatter plots of xCell immune scores and PD1 pathway targeting scores in the discovery and validation datasets. Regression lines with confidence intervals of 0.95 are shown for the long-term (blue) and short-term (red) groups.

In [ ]:
# B) Scatter plots of xCell immune scores and PD1 pathway targeting scores in the discovery and validation datasets. Regression lines with confidence intervals of 0.95 are shown for the long-term (blue) and short-term (red) groups.
scatterplot_indegree_cellTypeScore<-function(xcell_exp, pd1_deg, cellType, datasetName){
  celltype_score<-xcell_exp[c(cellType,"survival")] 
  celltype_info<-merge(celltype_score, as.data.frame(t(pd1_deg)), by = "row.names")
  celltype_info$pd1_mean<-rowMeans(celltype_info[4:(length(colnames(celltype_info)))])
  plot <-  ggscatter(celltype_info,x="pd1_mean", y="ImmuneScore", color = "survival", add = "reg.line", conf.int = TRUE ,palette = c("dodgerblue", "firebrick1"))+stat_cor(aes(color = survival)) + labs(title = datasetName, x = "PD1 pathway targeting score", y= "xCell Immune Score")
  plot
}

immune_score_scatter1<- scatterplot_indegree_cellTypeScore(dat1_xcell, dat1_pd1_deg, "ImmuneScore", "Discovery dataset 1")
immune_score_scatter2<- scatterplot_indegree_cellTypeScore(dat2_xcell, dat2_pd1_deg, "ImmuneScore", "Discovery dataset 2")
immune_score_scatter3<- scatterplot_indegree_cellTypeScore(dat3_xcell, dat3_pd1_deg, "ImmuneScore", "Validation dataset")

In [ ]:
# Plot supplemental figure 2
ggarrange(immune_score_box1, immune_score_box2, immune_score_box3, immune_score_scatter1, immune_score_scatter2, immune_score_scatter3, ncol = 3, nrow = 2)

## 4. Association with mutation load

Here, we will correlate PD1 targeting scores with overall mutation load, as well as with mutation scores of different biological pathways, to determine if there is an association between PD1 regulation and somatic mutations. This code will reproduce Figure 5<sup>1</sup>

### 4.1. Visualize PD1 pathway gene targeting scores against overall number of mutations per patient

Here, we will reproduce Figure 5A<sup>1</sup>. We will start by loading all degrees (Gene Targeting Scores) from discovery dataset 1

In [ ]:
load("/opt/data/netZooR/gbm/dat2_degree.RData")

Get PD1 genes from the Reactome PD1 signature that was used in the GSEA analyses on the gene targeting scores

In [ ]:
reactome_signature_file <- "/opt/data/netZooR/gbm/c2.cp.reactome.v5.0.symbols.gmt"
reactome_signatures <- read.table(reactome_signature_file, header = FALSE, sep = "\t", col.names = paste0("V",seq_len(1000)), fill = TRUE)
row.names(reactome_signatures) <- reactome_signatures[,1]
reactome_signatures <- reactome_signatures[,3:ncol(reactome_signatures)]
reactome_signatures <- as.matrix(reactome_signatures)
pd1idx <- grep("PD1", row.names(reactome_signatures))
pd1genes <- reactome_signatures[pd1idx,]
pd1genes <- pd1genes[which(pd1genes!="")] # 18 genes

Calculate the sum of PD1 gene degrees per patient

In [ ]:
pd1d <- indegree[which(row.names(indegree) %in% pd1genes),] # 13/18 genes available
pd1ds <- apply(pd1d, 2, sum) # Sum of PD1 degrees for 522 patients in discovery dataset 1

Calculate the total mutation burden (based on all genes) for each patient. For this, we  will read in the original .maf file from TCGA that was prepared in SAMBAR<sup>2</sup>

In [ ]:
load("/opt/data/netZooR/gbm/data_all_byMutation.Rdat")

Remove unknown genes and silent mutations

In [ ]:
maf.data <- maf.data[which(maf.data$hugo_symbol!="Unknown"),]
maf.data <- maf.data[which(maf.data$variant_classification!="Silent"),]

Sum the number of mutations per gene, per patient

In [ ]:
maf.data <- maf.data[,c(1,4:7,16)] # Get relevant columns
maf.data <- unique(maf.data) # Filter mutations present on the same location in the same patient, we only need to count these once
nrow(maf.data)==nrow(unique(maf.data[,-2])) # Test is there are duplicates on the different reference genomes for the same patient/mutation. There are none, so we will not need to merge these
maf.data <- maf.data[,-2]

tumsample <- as.character(maf.data[,5])
idx <- which(substr(tumsample, 14, 15) %in% c("01", "03")) # Select primary tumors only
maf.data <- maf.data[idx,c(1,5)] # This is the final .maf file that we can use to calculate the total number of mutations in each gene, per patient. We will do this below using the "igraph" package

g=graph.data.frame(maf.data)
mat <- get.adjacency(g,sparse=FALSE)
npat <- length(unique(maf.data[,2]))
ngene <- length(unique(maf.data[,1]))
mat <- mat[1:ngene, (ngene+1):ncol(mat)]
mpat <- substr(colnames(mat),1,12)
length(which(duplicated(mpat))) # There are some 71 "duplicated" entries
duppat <- mpat[which(duplicated(mpat))]

gbmpat <- colnames(indegree)
length(which(gbmpat %in% gsub("-", "\\.", tolower(duppat)))) # None of the glioblastoma patients are have duplicated data
colnames(mat) <- gsub("-", "\\.", tolower(substr(colnames(mat), 1, 12)))
mat <- mat[,which(colnames(mat) %in% gbmpat)]
nmut <- apply(mat, 2, sum) # Total number of mutations per patient

Visualize PD1 pathway gene targeting scores against overall number of mutations per patient (Figure 5A<sup>1</sup>)

In [ ]:
length(which(names(pd1ds) %in% names(nmut))) # 235
pd1ds_sub <- pd1ds[which(names(pd1ds) %in% names(nmut))]
pd1ds_sub <- pd1ds_sub/length(which(row.names(indegree) %in% pd1genes)) # Average GTS for PD1 pathway
all(names(pd1ds_sub)==names(nmut))
# pdf("pd1_avg_degree_score_nmut.pdf", w=3.5, h=4)
smoothScatter(cbind(pd1ds_sub, nmut), xlab="PD1 pathway targeting score", ylab="Mutation load")
# dev.off()

### 4.2. Correlate PD1 pathway targeting score with pathway mutation scores

Here, we will compare the PD1 pathway targeting score, as well as the gene targeting scores of individual genes in the PD1 pathway, with pathway mutation scores, which we calculate using SAMBAR<sup>2</sup>

In [ ]:
pd1d_sub <- pd1d[,which(names(pd1d) %in% names(nmut))]

Get PD1 pathway score (divide sum by number of genes in the pathway)

In [ ]:
pd1ds <- pd1ds/length(which(row.names(indegree) %in% pd1genes))

Calculate SAMBAR mutation scores based on all genes

In [ ]:
signatureset <- "/opt/data/netZooR/gbm/c2.cp.v5.0.symbols.gmt"
data(exon.size)
genes <- row.names(mat)
edg <- sambar.convertgmt(signature=signatureset, cagenes=genes)
data(exon.size)
mutlength <- sambar.corgenelength(x=t(mat), cagenes=genes, exonsize=exon.size) # 235 2188
mutlength <- t(mutlength)
patmutrate <- apply(mutlength, 2, sum)

Correlate PD1 pathway targeting scores with the overall mutation score from SAMBAR

In [ ]:
cor.test(pd1ds_sub,patmutrate, method="spearman") # R=0.0156, p=0.8116

Now correlate the gene targeting score of the individual genes in the PD1 pathway with the overall SAMBAR mutation score

In [ ]:
pd1d_sub <- pd1d[,which(names(pd1d) %in% names(nmut))]
cor_sep_genes <- matrix(NA, nrow(pd1d_sub), 2)
row.names(cor_sep_genes) <- row.names(pd1d_sub)
colnames(cor_sep_genes) <- c("R","pval")
patmutrate <- patmutrate[which(names(patmutrate) %in% colnames(pd1d_sub))]
all(names(patmutrate)==colnames(pd1d_sub))
for(d in 1:nrow(cor_sep_genes)){
	cor_sep_genes[d,1] <- round(cor.test(unlist(c(pd1d_sub[d,])), patmutrate, method="spearman")$estimate,4)
	cor_sep_genes[d,2] <- round(cor.test(unlist(c(pd1d_sub[d,])), patmutrate, method="spearman")$p.val, 4)
}
cat("min R=", min(cor_sep_genes[,1]), "\nmax R=", max(cor_sep_genes[,1]), "\n")

### 4.3. Associate SAMBAR scores per pathway with PD1 targeting scores across patients

Here, we will use pathway mutation scores for individual patients from the original SAMBAR paper<sup>2</sup>, and associate these scores with PD1 targeting to reproduce Figure 5B<sup>1</sup>. This will answer the question whether mutations in a specific biological pathway associate with a higher PD1 targeting score. The SAMBAR mutation scores will be loaded here directly, but are also available on Zenodo (DOI: 10.5281/zenodo.1494861)

In [ ]:
load("/opt/data/netZooR/gbm/TCGA_SAMBAR.RData")

Normalize the SAMBAR pathway mutation scores by the total patient-specific mutation rate calculated across cancer-associated genes (as done in the SAMBAR paper<sup>2</sup>)

In [ ]:
mutationrate <- apply(gene_scores, 2, sum)
signt <- pathway_scores
for(i in 1:ncol(pathway_scores)){
	signt[,i] <- pathway_scores[,i]/mutationrate[i]
}

Correlate PD1 pathway targeting scores with pathway mutation scores from SAMBAR<sup>2</sup>

In [ ]:
colnames(signt) <- tolower(colnames(signt))
colnames(signt) <- gsub("-", "\\.", colnames(signt))
signt <- signt[,which(colnames(signt) %in% names(pd1ds_sub))] # Note that 3 patients are unavailable, as they do not have any mutations in the cancer-associated genes on which SAMBAR calculates its pathway mutation scores
pd1ds_sub <- pd1ds_sub[which(names(pd1ds_sub) %in% colnames(signt))]
all(colnames(signt)==names(pd1ds_sub)) # T
resmat <- matrix(NA, nrow(signt), 2)
colnames(resmat) <- c("R","pval")
row.names(resmat) <- row.names(signt)
for(i in 1:nrow(signt)){
	resmat[i,1] <- cor.test(pd1ds_sub, signt[i,])$estimate
	resmat[i,2] <- cor.test(pd1ds_sub, signt[i,])$p.val
}
resmat <- resmat[!is.na(resmat[,1]),]
summary(p.adjust(resmat[,2], method="BH")) # min is 0.06
resmat <- cbind(resmat, p.adjust(resmat[,2], method="BH"))

In [ ]:
head(resmat)
resmat <- resmat[order(resmat[,3]),]

In [ ]:
resmat <- cbind(resmat, abs(resmat[,1])) # absolute R values
resmat <- cbind(resmat, log(resmat[,2],10)) # log pvals
resmat <- cbind(resmat, log(resmat[,3],10)) # log adjp vals
colnames(resmat)[3] <- "adjp"
colnames(resmat)[4] <- "abs_Pearson_R"
colnames(resmat)[5] <- "log10_nominal_p"
colnames(resmat)[6] <- "log10_adjp"
colplot <- resmat[,1]
colplot[colplot<0] <- -1
colplot[colplot>0] <- 1
colplot2 <- rep(NA, length(colplot))
colplot2[which(colplot<0)] <- "#F2E631"
colplot2[which(colplot>0)] <- "#8D5B96"
#pdf("pearson_pd1targeting_mutpathways_dat2_significance_and_effect_size_clean.pdf", w=3.5, h=4)
plot(resmat[,c(6,4)], pch=16, ylim=c(0,0.3), col=colplot2, cex=1.5, xlab="log10FDR", ylab="Absolute Pearson R")
legend("topright", col=c("#8D5B96","#F2E631"), pch=16, legend=c("Pos","Neg"), bty="n")
#dev.off()

## 5. Differential methylation analysis
Differential methylation analysis using limma<sup>4</sup> on methylation B-values quantile normalized (limited to probes on PD1 pathway) to compare short- vs long-survival groups, and correcting for gender, adjuvant therapy and age.

The following function allows to perform differential methylation analysis using limma and correcting for the covariates: gender, adj therapy and age.

In [ ]:
methyl_limma<-function(obj,quantileNorm=TRUE){ 
  obj<-obj[,!is.na(pData(obj)$survival)]
  gene <- unlist(lapply(fData(obj)$pd1_genes, function(x) paste(x, collapse = ", ")))
  # Covariates
  surv <- factor(pData(obj)$survival, levels=c("long", "short"))
  gender <- as.character(pData(obj)$gender.y)
  gender[which(is.na(gender))] <- "NA"    
  gender <- as.factor(gender)
  adj <- as.character(pData(obj)$neoadjuvanttherapy)
  adj[which(is.na(adj))] <- "NA"    
  adj <- as.factor(adj)
  age <- as.numeric(pData(obj)$yearstobirth)
  age[which(is.na(age))] <- mean(age,na.rm=TRUE)
  # limma
  design <- model.matrix(~ gender + adj + age + surv)
  mat_lmFit<-exprs(obj)
  if(quantileNorm==TRUE) mat_lmFit<-normalizeQuantiles(mat_lmFit)
  fitGood <- lmFit(mat_lmFit,design)
  fitGood <- eBayes(fitGood)
  # Save output table
  deg <- topTable(fitGood,coef="survshort",number=Inf, genelist=gene)
  Mean <- t(apply(mat_lmFit, 1, function(x) by(x, surv, mean, na.rm=TRUE)))
  Mean <- Mean[rownames(deg),]
  tab <- data.frame("Gene.ID"=deg[,"ID"], "BetaMean.Short-term"=Mean[,"short"], "BetaMean.Long-term"=Mean[,"long"], deg[,c("P.Value","adj.P.Val")])
  tab
}

Run differential methylation analysis for 450K platform

In [ ]:
load("/opt/data/netZooR/gbm/methyl_obj450_pd1.RData")
Platform.450K <- methyl_limma(obj450_pd1,quantileNorm=TRUE)
head(Platform.450K)

Run differential methylation analysis for 27K platform

In [ ]:
load("/opt/data/netZooR/gbm/methyl_obj27_pd1.RData")
Platform.27K <- methyl_limma(obj27_pd1,quantileNorm=TRUE)
head(Platform.27K)

## 6. Validation of PD1 signaling genes in protein abundance
To validate protein abundance levels of PD1 signaling genes, we will download Reverse Phase Protein Array (RPPA) data from glioblastoma samples from TCGA, and subset these to samples from the same patients we used in our network analysis (n=233). We will then select the three PD1 signaling proteins for which abundance data is available. We will evaluate differential abundance of these proteins between the long-term and short-term survivors.

In [ ]:
# Downloaded data with RTCGA.RPPA
load("/opt/data/netZooR/gbm/rtcga_rppa_gbm.RData") # these data were downloaded from RTCGA.RPPA package on 2019-07-17, R version 3.5.2, RTCGA.RPPA_1.6.0, RTCGA_1.8.0
patinfo <- "/opt/data/netZooR/gbm/allpats_gbm0_gbm1.RData" # contains info on which patients belong to which survival group

Select protein abundance for PD1 pathway genes in primary tumors

In [ ]:
# dat <- GBM.RPPA # When using the RTCGA.RPPA package, this is how you get the glioblastoma data
row.names(dat) <- dat[,1]
dat <- dat[,-1]
dat <- t(dat)
idx <- which(substr(colnames(dat),14,15)!="01") # samples that are not primary tumors
dat <- dat[,-idx] # remove these
pd1genes <- c("Lck", "p62-LCK-ligand", "PDCD4") # these proteins are part of the PD1 pathway
dat <- dat[which(row.names(dat) %in% pd1genes),]
colnames(dat) <- gsub("-", "\\.", tolower(substr(colnames(dat),1,12))) # subset to TCGA patient IDs
dat <- dat[which(row.names(dat) %in% pd1genes),]
colnames(dat) <- gsub("-", "\\.", tolower(substr(colnames(dat),1,12)))

Load patient info per group

In [ ]:
load(patinfo)
pd1_rppa_gbm0 <- dat[,which(colnames(dat) %in% gbm0)] # long-term survival group
pd1_rppa_gbm1 <- dat[,which(colnames(dat) %in% gbm1)] # short-term survival group

Perform t-test and correct for multiple testing

In [ ]:
tstatres <- matrix(NA, nrow=nrow(dat), ncol=2)
colnames(tstatres) <- c("t", "pval")
for(d in 1:nrow(dat)){
	tstatres[d,1] <- t.test(pd1_rppa_gbm0[d,], pd1_rppa_gbm1[d,])$statistic
	tstatres[d,2] <- t.test(pd1_rppa_gbm0[d,], pd1_rppa_gbm1[d,])$p.val
}
p.adjust(tstatres[,2], method="BH")

Make violin plot

In [ ]:
test <- list(pd1_rppa_gbm0[3,],pd1_rppa_gbm1[3,])
test0 <- cbind(c(test[[1]]), "long-term")
test1 <- cbind(c(test[[2]]), "short-term")
test <- rbind(test0,test1)
test <- as.data.frame(test)
test[,1] <- as.numeric(as.character(test[,1]))
test[,2] <- as.factor(as.character(test[,2]))
colnames(test) <- c("abundance","survival")
newscale <- c("#6a99f5","#f8766d")
p <- ggplot(test, aes(x=survival, y=abundance, fill=survival))+
geom_violin()
#pdf("violin_p62-lck-ligand.pdf", w=4,h=3)
p+scale_fill_manual(values=newscale)+geom_boxplot(width=0.1)+theme_bw()+labs(x="", y="p62-Lck-ligand abundance")
#dev.off()

## References

1- Lopes-Ramos, Camila Miranda, et al. "Regulation of PD1 signaling is associated with prognosis in glioblastoma multiforme." bioRxiv (2021).

2- Kuijjer, Marieke Lydia, et al. "Cancer subtype identification using somatic mutation data." Br J Cancer (2018) May;118(11):1492-1501.

3- Aran, Dvir, Zicheng Hu, and Atul J. Butte. "xCell: digitally portraying the tissue cellular heterogeneity landscape." Genome biology 18.1 (2017): 1-14.

4- Ritchie, Matthew E., et al. "limma powers differential expression analyses for RNA-sequencing and microarray studies." Nucleic acids research 43.7 (2015): e47-e47.

5- Padi, Megha, and John Quackenbush. "Detecting phenotype-driven transitions in regulatory network structure." NPJ systems biology and applications 4.1 (2018): 1-12.